In [1]:
import os 
from langchain_openai import ChatOpenAI

from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnableMap
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.agents.agent_toolkits import GmailToolkit
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.document_loaders.excel import UnstructuredExcelLoader

## 1. Create a model

In [2]:
# from langchain.llms import Ollama
# # llm = Ollama(base_url = "http://localhost:11434",model = "llama2")

In [3]:
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
gmail_toolkit = GmailToolkit()
llm=ChatOpenAI(temperature=0.0,model="gpt-3.5-turbo")


### 2. Load the documents and split it

In [4]:
# from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
# import google.generativeai as genai
# from langchain_google_genai import ChatGoogleGenerativeAI

# genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
# generation_config = {
#     "temperature": 0.9,
#     "top_k": 2,
# }
# llm = ChatGoogleGenerativeAI(model = "gemini-pro", generation_config =generation_config)


In [5]:

# Load the document
file_path = "info.xlsx"
loader = UnstructuredExcelLoader(file_path)
docs = loader.load()
# Split the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
splits = text_splitter.split_documents(docs)

# embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# To database

db = Chroma.from_documents(documents= splits, embedding = embeddings)
retreiver = db.as_retriever()

/Volumes/Ajeet/EmailGenerator/EmailGen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Create embeddings and retriever

### 4. Retrieving mails from ajeetacharya02@gmail.comm

In [33]:
output = {"emails":{
    "sender": "john.doe@example.com",
    "subject": "delivery charge?",
    "content": "What is the delivery charge?",
  }
}
type(output)

dict

In [39]:
def check_for_mail():
    prompt =f"""You have access to an external system that can check email. Please use it to see if I have received a new email. If there are any, filter out non-essential ones such as newsletters, Blogs, promotional content and notifications.Use your expertise in email content analysis to distinguish emails from the rest, pay attention to the sender and avoid invalid emails. Return the response as a DICT containing the sender's email address,the subject line of each email and the contents in the email,  separated by commas without double inverted commas in the output."""

    agent = initialize_agent(agent= AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                            tools= gmail_toolkit.get_tools(), 
                            llm= llm, 
                            verbose = True, 
                            max_execution_time = 1600,
                            max_iterations = 1000)
    
    # prompt = PromptTemplate(template=template)
    response = agent.run(prompt)
    return response

In [40]:
response = check_for_mail()
response



> Entering new AgentExecutor chain...
I will search your email for new messages and filter out non-essential ones based on the sender and content. Let's retrieve and analyze the emails to provide you with the relevant information.

Action:
```
{
  "action": "search_gmail",
  "action_input": {
    "query": "is:unread",
    "resource": "messages",
    "max_results": 10
  }
}
```


Observation: [{'id': '18effb8d47a52aa4', 'threadId': '18effb8d47a52aa4', 'snippet': 'Hello, how can i place my order?', 'body': 'Hello, how can i place my order?\r\n', 'subject': 'CHITCHAT', 'sender': 'roshni thapaa <roshnithapaa@gmail.com>'}, {'id': '18eff27a6deab9fb', 'threadId': '18eff27a6deab9fb', 'snippet': 'i want to place order.', 'body': 'i want to place order.\r\n', 'subject': 'place', 'sender': 'roshni thapaa <roshnithapaa@gmail.com>'}, {'id': '18eff22b6a9477f1', 'threadId': '18eff22b6a9477f1', 'snippet': 'I want to deliver myself to japan', 'body': 'I want to deliver myself to japan\r\n', 'subject'

{'emails': [{'sender': 'roshnithapaa@gmail.com',
   'subject': 'CHITCHAT',
   'content': 'Hello, how can i place my order?'},
  {'sender': 'roshnithapaa@gmail.com',
   'subject': 'place',
   'content': 'i want to place order.'},
  {'sender': 'roshnithapaa@gmail.com',
   'subject': 'FLIGHT',
   'content': 'I want to deliver myself to japan'},
  {'sender': 'roshnithapaa@gmail.com',
   'subject': 'flight',
   'content': 'I want to go to japan'},
  {'sender': 'ajeetach97701@gmail.com',
   'subject': 'Delivery Charge',
   'content': 'What is the delivery cost?'}]}

In [34]:
type(response)

str

In [10]:
emails

[{'sender': 'roshnithapaa@gmail.com',
  'subject': 'CHITCHAT',
  'content': 'Hello, how can i place my order?'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'place',
  'content': 'i want to place order.'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'FLIGHT',
  'content': 'I want to deliver myself to japan'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'flight',
  'content': 'I want to go to japan'},
 {'sender': 'ajeetach97701@gmail.com',
  'subject': 'Delivery Charge',
  'content': 'What is the delivery cost?'}]

In [59]:
prompt_template = """ {context} As a representative, I need to write an email regarding your order in email response. 
Most emails can be categorized into only one type: Orders
Do Not respond to the email if it is not related to ORDER.
Please write a professional reponse email based on the customer email.
DONOT EXCEED MORE THAN 30 WORDS.
query:```{question}```
context:```{context}```
Provide reply only without backticks and nothing else.
"""


In [60]:
prompt = """

You are a virtual assistant who answers users query. When a user asks a question  
you have to answer him based on the context given in three back ticks and make sure to provide each and every details.
context: ```{context}```
question is given in double back ticks : ``{question}``.

If the question is out of context, do not provide the wrong answer just say i am sorry i don't have an ansawer to that question.

"""
template = ChatPromptTemplate.from_template(prompt_template)
output_parser = StrOutputParser()
chain = RunnableMap(
    {
        "context": lambda x:db.similarity_search(x['question'], k =1),
        "question": lambda x:x['question']
    }
)|template|llm|output_parser


In [23]:
response.get('emails')

[{'sender': 'roshnithapaa@gmail.com',
  'subject': 'CHITCHAT',
  'content': 'Hello, how can i place my order?'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'place',
  'content': 'i want to place order.'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'FLIGHT',
  'content': 'I want to deliver myself to japan'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'flight',
  'content': 'I want to go to japan'},
 {'sender': 'ajeetach97701@gmail.com',
  'subject': 'Delivery Charge',
  'content': 'What is the delivery cost?'}]

In [16]:
email_data = response['emails']
email_data

[{'sender': 'roshnithapaa@gmail.com',
  'subject': 'CHITCHAT',
  'content': 'Hello, how can i place my order?'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'place',
  'content': 'i want to place order.'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'FLIGHT',
  'content': 'I want to deliver myself to japan'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'flight',
  'content': 'I want to go to japan'},
 {'sender': 'ajeetach97701@gmail.com',
  'subject': 'Delivery Charge',
  'content': 'What is the delivery cost?'}]

In [20]:
to_send = email_data[1]

{'sender': 'roshnithapaa@gmail.com',
 'subject': 'place',
 'content': 'i want to place order.'}

In [ ]:
send_email(to_email, subject, message):

In [56]:
import os, smtplib
import email.message as em


def send_email(to_email, subject, message):
    smtp_server = os.getenv('SMTP_HOST')
    smtp_port = int(os.getenv('SMTP_PORT'))  # Convert to int
    sender_email = os.getenv('EMAIL_DEV')
    password = os.getenv('EMAIL_PASSWORD')

    msg = em.Message()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(message)

    smtp = smtplib.SMTP(smtp_server, smtp_port)
    smtp.starttls()
    smtp.login(sender_email, password)
    smtp.sendmail(msg['From'], [msg['To']], msg.as_string())
    smtp.quit()
    print("Successfully sent email to", to_email)


In [57]:
for email in email_data:
    print(email['content'])
    result = chain.invoke({"question":email['content']})
    send_email(to_email=email['sender'], subject=email['subject'], message=result)
    print(result)

Hello, how can i place my order?
Successfully sent email to roshnithapaa@gmail.com
I am sorry, I don't have an answer to that question.
i want to place order.
Successfully sent email to roshnithapaa@gmail.com
I am sorry, I don't have an answer to that question.
I want to deliver myself to japan
Successfully sent email to roshnithapaa@gmail.com
I am sorry, I don't have an answer to that question.
I want to go to japan
Successfully sent email to roshnithapaa@gmail.com
I am sorry I don't have an answer to that question.
What is the delivery cost?
Successfully sent email to ajeetach97701@gmail.com
```
What is the delivery cost?
The delivery charge is $3.
```


In [44]:
import os, smtplib
import email.message 


def send_email(to_email, subject, message):
    smtp_server = os.getenv('SMTP_HOST')
    smtp_port = int(os.getenv('SMTP_PORT'))  # Convert to int
    sender_email = os.getenv('EMAIL_DEV')
    password = os.getenv('EMAIL_PASSWORD')

    msg = email.message.Message()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(message)

    smtp = smtplib.SMTP(smtp_server, smtp_port)
    smtp.starttls()
    smtp.login(sender_email, password)
    smtp.sendmail(msg['From'], [msg['To']], msg.as_string())
    smtp.quit()
    print("Successfully sent email to", to_email)
